In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import yaml
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd
import traceback

In [3]:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)

In [4]:
with open('config.yaml') as f:
    var = yaml.safe_load(f)

In [5]:
driver.get("https://linkedin.com/uas/login")

In [6]:
driver.find_element("xpath", '//*[@id="username"]').send_keys(var['username'])
driver.find_element("xpath", '//*[@id="password"]').send_keys(var['password'])
time.sleep(1)

In [7]:
driver.find_element(
    "xpath", '//*[@id="organic-div"]/form/div[3]/button').click()

In [8]:
# go to job page, the url here is data scientist job posting. Replace with the job you want to scrape.
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=2903306264&f_E=2%2C3&f_JT=F&f_TPR=r604800&keywords=machine%20learning%20engineer&refresh=true")

In [9]:
links = []
print('Links collecting now.')
try:
    for page in tqdm(range(2, 5)): 

        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CLASS_NAME, 'jobs-search-results__list-item')
        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links:
                    links.append(a.get_attribute('href'))
                else:
                    pass
            driver.execute_script("arguments[0].scrollIntoView();", job)
        driver.find_element(
            "xpath", f"//button[@aria-label='Page {page}']").click()
except Exception:
    traceback.print_exc()
print('Found ' + str(len(links)) + ' links for job offers')

Links collecting now.


100%|██████████| 3/3 [00:18<00:00,  6.13s/it]

Found 73 links for job offers


In [10]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = []
job_desc = []

i = 0
j = 1

for url in tqdm(links):
    time.sleep(1)
    try:
        driver.get(url)
        driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[2]/footer').click()
    except Exception:
        pass
    try:
        driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[4]/footer').click()
    except:
        pass

    try:
        job_titles.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/h1').text)
    except Exception:
        job_titles.append('unknown')
    try:
        company_names.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[1]/span[1]').text)
    except Exception:
        company_names.append('unknown')
    try:
        company_locations.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[1]/span[2]').text)
    except Exception:
        company_locations.append('unknown')
    try:
        work_methods.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[1]/span[3]').text)
    except Exception:
        work_methods.append('unknown')
    try:
        post_dates.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[2]/span[1]').text)
    except Exception:
        post_dates.append('unknown')
    try:
        work_times.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[2]/ul/li[1]').text)
    except Exception:
        work_times.append('unknown')
        
    try:
        job_description = driver.find_elements(
            By.CLASS_NAME, 'jobs-description__content')
        for description in job_description:
            job_text = description.find_element(
                By.CLASS_NAME, "jobs-box__html-content").text
            job_desc.append(job_text)
    except:
        job_desc.append('No description found')

    j += 1

100%|██████████| 73/73 [07:10<00:00,  5.90s/it]


In [11]:
df = pd.DataFrame(list(zip(job_titles, company_names,
                           company_locations, work_methods,
                           post_dates, work_times, links, job_desc)),
                  columns=['job_title', 'company_name',
                           'company_location', 'work_method',
                           'post_date', 'work_time', 'job_links', 'job_desc'])


In [13]:
d = {"minutes ago": 1,"minute ago": 1, "hour ago" : 60, "hours ago": 60,"day ago" : 3600,"days ago": 3600,"week ago": 4000,"weeks ago": 4500,"unknown":10000}
df ["sortvalue"] = 1
for i in range(len(df)):
    x = df.iloc[i,4].split(" ",1)
    if (len(x)==1):
        x[0] = 10000
        x.append( "unknown")
    df.iloc[i,-1] = int(x[0])*d[x[1]]

df = df.sort_values('sortvalue')

In [17]:
with pd.ExcelWriter('../data/jobs.xlsx') as job :
    for i in range(0,len(df),75):
        df[i:i+75].to_excel(job, sheet_name='Sheet_' + str(i // 75))

In [18]:
df.head()

,job_title,company_name,company_location,work_method,post_date,work_time,job_links,job_desc,sortvalue
3,Software Engineer,AstraZeneca,"Gaithersburg, MD",unknown,2 hours ago,Full-time · Associate,https://www.linkedin.com/jobs/view/3574907169/...,"About the job\nAre you experienced in, and pas...",120
33,"Software Engineer, Web",Cboe Global Markets,"Chicago, IL",unknown,2 hours ago,"$105,000/yr - $158,700/yr (from job descriptio...",https://www.linkedin.com/jobs/view/3572517162/...,About the job\nJob Description\n\nWhat We Do\n...,120
12,AI & HPC Application Engineer,Inspur Systems,"Fremont, CA",On-site,3 hours ago,Full-time · Entry level,https://www.linkedin.com/jobs/view/3570835228/...,"About the job\nInspur Systems Inc., located in...",180
27,"Research Software Engineer, AI",Spotter,"Los Angeles, CA",Hybrid,4 hours ago,Full-time · Entry level,https://www.linkedin.com/jobs/view/3574578027/...,About the job\nOverview\n\nSpotter empowers to...,240
5,Algorithm Engineer,neuro42,San Francisco Bay Area,On-site,4 hours ago,Full-time,https://www.linkedin.com/jobs/view/3577987482/...,About the job\nThe next evolution of MRI is he...,240
